1- Importing requests it allows you to send HTTP requests, without the need for manual labor. There's no need to manually add query strings to your URLs.

2- Importing getpass for a secure password prompt

3- Json is the exclusive format GitHub API sends data.

In [2]:
import requests
#from getpass import getpass
import json
from getpass import getpass
import pandas as pd
from github import Github
import networkx as nx
import numpy as np
import ast
import datetime

Acquiring OAuth Token.. 

Quick reminder: OAuth is an open standard for access delegation, commonly used as a way for Internet users to grant websites or applications access to their information on other websites but without giving them the passwords.

In [23]:
token='b8b7ceb4f80063cc2803389b32368e735b177143'

In [24]:
username = 'YoussefEssDS' #put ur Github user name here! 
password = 'Seek greatness1' #put ur Github password name here! 
#Credentials are transmitted over a secure SSL connection
url = 'https://api.github.com/authorizations'
note = 'Aquiring_token'
post_data = {'scopes':['repo'],'note': note }
response = requests.post(url,auth = (username, password),data = json.dumps(post_data),) # Mohim this response needs input in this format dunno why!
print("API response:", response.text)
#print("Your OAuth token is", response.json()['token'])


API response: {"message":"Validation Failed","errors":[{"resource":"OauthAccess","code":"already_exists","field":"note"}],"documentation_url":"https://developer.github.com/v3/oauth_authorizations/#create-a-new-authorization"}


For now we'll try to list a user's followers.

As you can see we choosed the Tensorflow repository as a target for our study.

In [36]:
repo_user='tensorflow'
repo='tensorflow'

We will extract monthly data for contribution to the tensorflow repository between 01/2019 & 06/2019.

GitHub identifies contributors by author email address. This endpoint groups contribution counts by GitHub user, which includes all associated email addresses. To improve performance, only the first 500 author email addresses in the repository link to GitHub users. The rest will appear as anonymous contributors without associated GitHub user information.

In [37]:
from_initial_followship_dict={}
to_initial_followship_dict={}
client = Github(token,per_page=100)
user = client.get_user(repo_user)
repo = user.get_repo(repo)

In [65]:
def createContrib(s,u):
    contributions = [ s for s in repo.get_commits(since=s,until=u)]
    contributors=[s.author for s in contributions]
    contributors=list(dict.fromkeys(contributors))
    #Some commits belong to users that don't have an account anymore, or changed it so when inquire the user would be NonType we have to eliminate those.
    contributors=[s for s in contributors if s!=None ]
    return(contributors)

In [66]:
def createFollwDict(contributors):
    for user in contributors:
        from_initial_followship_dict[user.login]=[follower for follower in user.get_followers()]
    for user in contributors:
        to_initial_followship_dict[user.login]=[followed for followed in user.get_following()]
    return(from_initial_followship_dict,to_initial_followship_dict)

In [86]:
def createFiArray(fromFollDict,toFollDict,date):
    l=np.ndarray(shape=(0,4),dtype=str)
    #Followers of contributors:
    for contributor in fromFollDict.keys():
        for follower in fromFollDict[contributor]:
            l=np.append(l,[[date,contributor,follower.login,"contributer"]],axis=0)

    #The followed by contributors:
    for contributor in toFollDict.keys():
        for followed in toFollDict[contributor]:
            if followed in toFollDict.keys():
                l=np.append(l,[[date,followed.login,contributor,"contributer"]],axis=0)
            else:
                l=np.append(l,[[date,followed.login,contributor,"non-contributer"]],axis=0)
    return(l)

In [67]:
contributors1=createContrib(datetime.datetime(2019,1,1),datetime.datetime(2019,2,1))
contributors2=createContrib(datetime.datetime(2019,2,1),datetime.datetime(2019,3,1))    
contributors3=createContrib(datetime.datetime(2019,3,1),datetime.datetime(2019,4,1))    
contributors4=createContrib(datetime.datetime(2019,4,1),datetime.datetime(2019,5,1))    
contributors5=createContrib(datetime.datetime(2019,5,1),datetime.datetime(2019,6,1))    

In [68]:
fromFollDict1,toFollDict1=createFollwDict(contributors1)
fromFollDict2,toFollDict2=createFollwDict(contributors2)
fromFollDict3,toFollDict3=createFollwDict(contributors3)
fromFollDict4,toFollDict4=createFollwDict(contributors4)
fromFollDict5,toFollDict5=createFollwDict(contributors5)

In [87]:
l1=createFiArray(fromFollDict1,toFollDict1,'01-2019')
l2=createFiArray(fromFollDict2,toFollDict2,'02-2019')
l3=createFiArray(fromFollDict3,toFollDict3,'03-2019')
l4=createFiArray(fromFollDict4,toFollDict4,'04-2019')
l5=createFiArray(fromFollDict5,toFollDict5,'05-2019')

In [90]:
l=np.concatenate((l1,l2,l3,l4,l5))

In [91]:
followship_data_frame=pd.DataFrame(data=l,columns=['Date','To','From','To is contributor?'])
followship_data_frame.drop_duplicates()

,Date,To,From,To is contributor?
0,01-2019,trevor-m,Byron0221,contributer
1,01-2019,trevor-m,ShayanPersonal,contributer
2,01-2019,trevor-m,jonike,contributer
3,01-2019,trevor-m,nazifberat,contributer
4,01-2019,trevor-m,JaredYeDH,contributer
5,01-2019,trevor-m,iancormac84,contributer
6,01-2019,trevor-m,asetsuna,contributer
7,01-2019,trevor-m,sonej,contributer
8,01-2019,trevor-m,Vishal2188,contributer
9,01-2019,trevor-m,shainaprasad,contributer


Let's save the dataframe in a csv file to avoid extracting everytime.

In [92]:
pd.DataFrame.to_csv(followship_data_frame,'MainDF.csv')

In [27]:
followship_data_frame=pd.read_csv('MainDF.csv')
followship_data_frame=followship_data_frame.drop('Unnamed: 0',axis=1)

In [28]:
followship_data_frame

,Date,To,From,To is contributor?
0,01-2019,trevor-m,Byron0221,contributer
1,01-2019,trevor-m,ShayanPersonal,contributer
2,01-2019,trevor-m,jonike,contributer
3,01-2019,trevor-m,nazifberat,contributer
4,01-2019,trevor-m,JaredYeDH,contributer
5,01-2019,trevor-m,iancormac84,contributer
6,01-2019,trevor-m,asetsuna,contributer
7,01-2019,trevor-m,sonej,contributer
8,01-2019,trevor-m,Vishal2188,contributer
9,01-2019,trevor-m,shainaprasad,contributer


Let's attempt a visualization of the **intrest network for the contributers** of the Tensorflow repository using **Networkx**, afterwards we will use the open source software **Cytoscape** to visualize the same network.

In [5]:
G=nx.Graph()

In [40]:
len(followship_data_frame)

290540

In [39]:
#Let's create this graph:
for i in range (0,len(followship_data_frame)):
    if (followship_data_frame.loc[followship_data_frame.index==i,'To'].get_values()[0]=='contributor'):
        edge=(followship_data_frame.loc[followship_data_frame.index==i,'From'].get_values()[0], followship_data_frame.loc[followship_data_frame.index==i,'To'].get_values()[0])
        G.add_edge(*e)

str